In [1]:
from itertools import combinations
import numpy as np
import os
import pandas as pd
import random

In [2]:
POS_STARTERS = (("Q", 2), ("R", 2), ("W", 2), ("T", 1), (("R", "W", "T"), 2))
POS_ALL = (("Q", 3), ("R", 6), ("W", 6), ("T", 1))
MONEY_COLS = ["$VORP", "$MG", "$VOLS", "$ESPN"]
TOTAL_MONEY = 200

### Find cheat sheet in most recent directory

In [3]:
# Get folder for most recent season
folders = [
    f
    for f in os.listdir()
    if os.path.isdir(f) and not f.startswith(".") and not f.startswith("_")
]
newest_folder = sorted(folders)[-1]
assert "cheat_sheet.csv" in os.listdir(newest_folder), "No cheat sheet found"

### Preprocess/clean up cheat sheet

In [4]:
# Load cheat sheet
cheat_sheet = pd.read_csv(f"{newest_folder}/cheat_sheet.csv", header=5)

# Remove unnamed columns
cheat_sheet = cheat_sheet.loc[:, ~cheat_sheet.columns.str.contains("^Unnamed")]

# # Split all column names on periods and take the first part
cheat_sheet.columns = cheat_sheet.columns.str.split(".").str[0]

# Melt on duplicate columns
cols = cheat_sheet.columns[~cheat_sheet.columns.duplicated()]
n_splits = cheat_sheet.shape[1] // len(cols)
splits = np.split(np.arange(cheat_sheet.shape[1]), n_splits)
dfs = [cheat_sheet.iloc[:, split] for split in splits]
cheat_sheet = pd.concat(dfs, axis=0).reset_index(drop=True)

# Only take first character for values in POS column
cheat_sheet["POS"] = cheat_sheet["POS"].str[0]

# Remove unneeded positions
pos_needed = set(
    np.hstack(np.array([p[0] for p in POS_STARTERS], dtype=object)).tolist()
)
cheat_sheet = cheat_sheet[cheat_sheet["POS"].isin(pos_needed)]

### Load and integrate auction values and projections

In [5]:
# Load auction values
auction_values = pd.read_csv(f"{newest_folder}/espn_auction_values.csv")

# Load projections
projections = pd.read_csv(f"{newest_folder}/projections.csv", header=1)
projections = projections.rename(columns={"Player": "NAME"})

# Get auction values
auction_vals_list = []
first_col = auction_values.iloc[:, 0]
for i, val in enumerate(first_col):
    if "," in val and "$" in first_col.iloc[i + 1]:
        # Make sure values are at least 1
        first_col.iloc[i + 1] = first_col.iloc[i + 1].replace("$0", "$1")
        # Make series for each player
        s = pd.Series(
            {"NAME": val.split(",")[0], "$ESPN": first_col.iloc[i + 1]}
        )
        auction_vals_list.append(s)
auction_vals = pd.DataFrame(auction_vals_list)

# Merge auction values with cheat sheet
cheat_sheet = cheat_sheet.merge(auction_vals, on="NAME")

# Merge projections with cheat sheet
cheat_sheet = cheat_sheet.merge(projections[["NAME", "All Pos"]], on="NAME")

# Remove duplicates
cheat_sheet = cheat_sheet.drop_duplicates("NAME")

# Remove dollar signs and convert to integers for $ columns
for col in MONEY_COLS:
    cheat_sheet[col] = cheat_sheet[col].str.replace("$", "").astype(int)

/var/folders/5j/h9q625yn13vfynsjc2bymdy00000gn/T/ipykernel_38557/3467556.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_col.iloc[i + 1] = first_col.iloc[i + 1].replace("$0", "$1")
/var/folders/5j/h9q625yn13vfynsjc2bymdy00000gn/T/ipykernel_38557/3467556.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_col.iloc[i + 1] = first_col.iloc[i + 1].replace("$0", "$1")
/var/folders/5j/h9q625yn13vfynsjc2bymdy00000gn/T/ipykernel_38557/3467556.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

### Determine good team splits

In [6]:
def generate_sums(bins, counts):
    """
    Recursively generates all possible combinations of elements from bins
    based on the provided counts and sums them.

    Args:
        bins (list of lists): Each inner list represents a bin containing
                              elements to choose from.
        counts (list of ints): Each integer represents the number of elements
                               to select from the corresponding bin.

    Returns:
        dict: A dictionary where the keys are the sums of the combinations
              and the values are the combinations themselves.
    """
    # Base case: If there's only one bin, generate all combinations from it.
    if len(bins) == 1:
        return {sum(comb): comb for comb in combinations(bins[0], counts[0])}

    # Recursive case: Split bins into two halves.
    mid = len(bins) // 2
    left_bins = bins[:mid]
    right_bins = bins[mid:]
    left_counts = counts[:mid]
    right_counts = counts[mid:]

    # Generate sums (combinations) for the left and right halves.
    left_sums = generate_sums(left_bins, left_counts)
    right_sums = generate_sums(right_bins, right_counts)

    # Combine the left and right sums by adding their totals.
    combined_sums = {}
    for l_sum, l_comb in left_sums.items():
        for r_sum, r_comb in right_sums.items():
            total_sum = l_sum + r_sum
            combined_sums[total_sum] = l_comb + r_comb

    return combined_sums


def find_combinations(bins, counts, target_sum, shuffle=False):
    """
    Finds the combination of elements from bins that has the sum closest to
    the target sum.

    Args:
        bins (list of lists): Each inner list represents a bin containing
                              elements to choose from.
        counts (list of ints): Each integer represents the number of elements
                               to select from the corresponding bin.
        target_sum (int or float): The target sum to get as close to as
                                   possible with the combination.
        shuffle (bool): If True, shuffle the elements within each bin before
                        generating combinations. This introduces randomness
                        and can lead to different results on different runs.

    Returns:
        tuple: The combination of elements that has the sum closest to the
               target sum.
    """
    # Shuffle the elements within each bin if shuffle is True.
    if shuffle:
        for bin in bins:
            random.shuffle(bin)

    # Generate all possible sums and their corresponding combinations.
    possible_sums = generate_sums(bins, counts)

    # Initialize variables to track the closest combination.
    closest_diff = float("inf")
    closest_combination = None

    # Iterate through each combination to find the one closest to the target.
    for total_sum, combination in possible_sums.items():
        diff = abs(total_sum - target_sum)  # Calculate the difference.
        # Update the closest combination if this one is closer to the target.
        if diff < closest_diff:
            closest_diff = diff
            closest_combination = combination

    return closest_combination

In [7]:
def find_roster(cheat_sheet, positions, constraint, budget):
    # Make copy of cheat sheet
    cheat_sheet = cheat_sheet.copy()

    # Split into bins
    bins, counts = [], []
    for pos, count in positions:
        if isinstance(pos, str):
            pos = [pos]
        bins.append(
            cheat_sheet[cheat_sheet["POS"].isin(pos)][constraint].values
        )
        counts.append(count)

    # Find closest combination
    closest_combination = find_combinations(bins, counts, budget, shuffle=True)

    # Get players
    player_idx = 0
    team = []
    for pos, count in positions:
        if isinstance(pos, str):
            pos = [pos]
        for _ in range(count):
            # Get player value
            val = closest_combination[player_idx]

            # Get players for this position
            pos_players = cheat_sheet[cheat_sheet["POS"].isin(pos)]

            # Find player in position with value
            player = pos_players[pos_players[constraint] == val].iloc[0]
            player["Slot"] = pos[0]

            # Add player to team
            team.append(player)

            # Remove player from pool
            cheat_sheet = cheat_sheet.drop(player.name)

            # Increment player index
            player_idx += 1
    return pd.DataFrame(team)

In [9]:
def split_into_starters_and_bench(team, pos_starters, objective):
    # Copy team
    team = team.copy()

    starters = []
    for pos, count in pos_starters:
        if isinstance(pos, str):
            pos = [pos]
        for _ in range(count):
            # Get players for this position
            pos_players = team[team["Slot"].isin(pos)]

            # Sort by objective
            pos_players = pos_players.sort_values(objective, ascending=False)

            # Grab player with highest objective and add to starters
            player = pos_players.iloc[0, :]
            starters.append(player)

            # Remove player from pool
            team = team.drop(player.name)

    return pd.DataFrame(starters), team

In [10]:
def find_best_team(
    cheat_sheet,
    pos_all,
    pos_starters,
    constraint,
    objective,
    budget,
    starters_weight=1,
    bench_weight=0.5,
    n_iter=1000,
):
    # Initialize variables to track the best team found
    best_team, best_total_objective = None, float("-inf")

    for _ in range(n_iter):
        # Find a team
        team = find_roster(cheat_sheet, pos_all, constraint, budget)

        # Split into starters and bench
        starters, bench = split_into_starters_and_bench(
            team, pos_starters, objective
        )

        # Calculate total objective
        total_objective = (
            starters_weight * starters[objective].sum()
            + bench_weight * bench[objective].sum()
        )

        # Update the best team if this one is better
        if total_objective > best_total_objective:
            best_team = team
            best_total_objective = total_objective

    starters, bench = split_into_starters_and_bench(
        best_team, pos_starters, objective
    )
    return starters, bench, best_total_objective

In [11]:
starters, bench, total_objective = find_best_team(
    cheat_sheet,
    POS_ALL,
    POS_STARTERS,
    "$ESPN",
    "All Pos",
    TOTAL_MONEY,
    n_iter=50,
)

In [12]:
money_cols_diff = [col.replace("$", "-$") for col in MONEY_COLS]
cheat_sheet[money_cols_diff] = cheat_sheet[MONEY_COLS].sub(
    cheat_sheet["$ESPN"], axis=0
)

In [13]:
# Add rank in money_cols_diff to cheat_sheet
cheat_sheet["-$VORP_Rank"] = cheat_sheet.groupby("POS")["-$VORP"].rank(
    ascending=False
)
cheat_sheet["-$MG_Rank"] = cheat_sheet.groupby("POS")["-$MG"].rank(
    ascending=False
)
cheat_sheet["-$VOLS_Rank"] = cheat_sheet.groupby("POS")["-$VOLS"].rank(
    ascending=False
)